In [157]:
import datetime as dt
import numpy as np
import pandas as pd
import pytz

from numba import jit
from matplotlib import pyplot as plt
%matplotlib inline

In [158]:
df = pd.read_hdf('./data_backtest_0.hdf').sort_values(by=['close_dt_pst','tsec'])
df['price_chg'] = df['close_next'] - df['close']
df['price_diff'] = df['price'] - df['close']
df['pct_diff'] = df['price_diff']/df['close']
df['close_dt_pst'] = df['close_dt_pst'].apply(lambda x: np.datetime64(x))
df['next_dt'] = df['next_dt'].apply(lambda x: np.datetime64(x))

In [159]:
df.head()

,close_dt_pst,tsec,next_dt,close,price,close_next,price_chg,price_diff,pct_diff
0,2021-03-03 19:33:00,35.0,2021-03-03 19:35:00,1570.71,1574.890688,1574.48,3.77,4.180688,0.002662
1,2021-03-03 19:33:00,35.0,2021-03-03 19:35:00,1570.71,1572.839180,1574.48,3.77,2.129180,0.001356
2,2021-03-03 19:35:00,15.0,2021-03-03 19:37:00,1574.48,1576.943434,1572.93,-1.55,2.463434,0.001565
3,2021-03-03 19:36:00,0.0,2021-03-03 19:38:00,1571.98,1574.888450,1572.30,0.32,2.908450,0.001850
5,2021-03-03 19:39:00,0.0,2021-03-03 19:41:00,1569.15,1573.588614,1565.59,-3.56,4.438614,0.002829


In [160]:
pct_buy = -0.005231
pct_flat = -0.002140
pct_sell = 0.000091
df_trades = pd.DataFrame(columns=['pos','open_time','close_time','open_price','close_price',
                                  'price_max','price_min','type'])
k = 0
df_trades.loc[k,'pos'] = 0
df_trades.loc[k,'price_min'] = df.loc[0,'price']
df_trades.loc[k,'price_min'] = df.loc[0,'price']

# column = np_df[:,0])
np_df = df[['tsec', 'price', 'pct_diff']].to_numpy()
close_dts = df['close_dt_pst'].to_numpy()

In [7]:
print(df.columns)
df_trades

Index(['close_dt_pst', 'tsec', 'next_dt', 'close', 'price', 'close_next',
       'price_chg', 'price_diff', 'pct_diff'],
      dtype='object')


,pos,open_time,close_time,open_price,close_price,price_max,price_min,type
0,0,NaN,NaN,NaN,NaN,NaN,1574.89,NaN


In [188]:
# @jit(nopython=True)
def run_bt(np_df, close_dts, pct_buy, pct_flat, pct_sell, num_reps, last_trade):
    '''
        End result is going to be 
        columns: position, open price, close price, type, open_pct_diff, close_pct_diff, open_tsecs, close_tsecs
    
    
    '''
    output = np.empty((num_reps, 8), np.float64)
    output[:] = np.nan
    output[0, 0] = 0
    open_dates = np.empty(num_reps)
    trade_types = np.empty(num_reps)
    
    for i in range(num_reps):
        trade_type=None
        row = np_df[i, :]
        close_dt = close_dts[i]
        pct_diff = row[-1]
        price = row[1]
        tsec = row[0]
        if pct_diff < pct_buy:
            if output[last_trade, 0] == 0:
                trade_type = 'open_long'
            elif output[last_trade, 0] == -1:
                trade_type = 'flip_long'
        elif pct_diff > pct_sell:
            if output[last_trade, 0] == 0:
                trade_type = 'open_short'
            elif output[last_trade, 0] == 1:
                trade_type = 'flip_short'

        elif output[last_trade, 0] == 1 and pct_diff > pct_flat:
            trade_type = 'close'
        elif output[last_trade, 0] == -1 and pct_diff < pct_flat:
            trade_type = 'close'

        if trade_type is not None:
            if trade_type in ['open_long','open_short']:
                last_trade = i
                if trade_type == 'open_long':
                    output[last_trade, 0] = 1
                    output[last_trade, 3] = 1
                else:
                    output[last_trade, 0] = -1
                    output[last_trade, 3] = -1
                
                output[last_trade, 1] = price
                output[last_trade, 4] = pct_diff
                output[last_trade, -2] = tsec                

            else:
                #first close and then open a new pos if we are flipping
                output[last_trade, 0] = 0
                output[last_trade, 2] = price
                output[last_trade, 5] = pct_diff
                output[last_trade, -1] = tsec                
                if trade_type in ['flip_long','flip_short']:
                    last_trade = i
                    if trade_type == 'flip_long':
                        output[last_trade, 0] = 1
                    else:
                        output[last_trade, 0] = -1
                    output[last_trade, 1] = price
                    output[last_trade, 4] = pct_diff
                    output[last_trade, -2] = tsec
                    
                    if trade_type == 'flip_long':
                        output[last_trade, 3] = 1
                    else:
                        output[last_trade, 3] = -1

    return output

In [189]:
rez = run_bt(np_df, close_dts, pct_buy, pct_flat, pct_sell, 50, 0)
pd.DataFrame(rez, columns=["position", "open_price", "close_price", 
                           "type", "open_pct_diff", "close_pct_diff", "open_tsecs", "close_tsecs"]).dropna()

,position,open_price,close_price,type,open_pct_diff,close_pct_diff,open_tsecs,close_tsecs
0,0.0,1574.890688,1548.770199,-1.0,0.002662,-0.005247,35.0,35.0
44,0.0,1548.770199,1553.645889,1.0,-0.005247,-0.002116,35.0,35.0
46,0.0,1558.372660,1552.232777,-1.0,0.000920,-0.003023,35.0,50.0
